In [ ]:
!pip install subgrounds
!pip install flipside
!pip install duckdb

In [ ]:
from subgrounds.subgrounds import Subgrounds
sg = Subgrounds()
hop_prtocol_transfers = sg.load_subgraph('https://gateway.thegraph.com/api/[api-key]/subgraphs/id/Cjv3tykF4wnd6m9TRmQV7weiLjizDnhyt6x2tTJB42Cy')

# **Receive transaction data**

In [ ]:
import pandas as pd
raw_data = pd.DataFrame()

transfers_first_100k = hop_prtocol_transfers.Query.transferSentToL2S(
  orderBy=hop_prtocol_transfers.TransferSentToL2.timestamp,
  orderDirection='asc',
  first=100000
)

first_100k_result = sg.query_df(transfers_first_100k).squeeze()


In [ ]:
frames = [raw_data, first_100k_result]
raw_data = pd.concat(frames, ignore_index=True)

In [5]:
chunk = 100000
while 1:
    next_chunk = hop_prtocol_transfers.Query.transferSentToL2S(
    orderBy=hop_prtocol_transfers.TransferSentToL2.timestamp,
    orderDirection='asc',
    skip=chunk,
    first=100000)
    next_chunk = sg.query_df(next_chunk).squeeze()
    if next_chunk.empty:
        break
    frames = [raw_data, next_chunk]
    raw_data = pd.concat(frames, ignore_index=True)
    chunk += 100000


In [7]:
raw_data = raw_data[['transferSentToL2S_timestamp',
                   'transferSentToL2S_transactionHash', 'transferSentToL2S_from',
                   'transferSentToL2S_amount', 'transferSentToL2S_token',
                   'transferSentToL2S_destinationChainId']]

# **Transform raw data**

In [8]:
from datetime import datetime

def obtain_chain_name(chain_id):
    if chain_id == 137:
        return "Polygon"
    elif chain_id == 10:
        return "Optimism"
    elif chain_id == 100:
        return "Gnosis"
    elif chain_id == 8453:
        return "Base"
    elif chain_id == 42161:
        return "Arbitrum One"
    elif chain_id == 42170:
        return "Arbitrum Nova"


def obtain_token_decimals(token_name):
    if token_name == "USDC" or token_name == "USDT":
        return pow(10, 6)
    elif token_name == "DAI" or token_name == "ETH" or token_name == "MATIC":
        return pow(10, 18)
    elif token_name == "WBTC":
        return pow(10, 8)


def fix_days(row):
    row[0] = row[0][0:10]
    return row

def convert_timestamp(unix_timestamp):
    ts = int(unix_timestamp)
    timestamp = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')
    return timestamp


In [9]:
final_df = pd.DataFrame()

In [ ]:
final_df["timestamp"] = raw_data["transferSentToL2S_timestamp"].apply(convert_timestamp)
final_df["tx_hash"] = raw_data["transferSentToL2S_transactionHash"]
final_df["sender"] = raw_data["transferSentToL2S_from"]
final_df["token_name"] = raw_data["transferSentToL2S_token"]

raw_amount = list(map(int, raw_data["transferSentToL2S_amount"].values.tolist()))
decimals = list(raw_data["transferSentToL2S_token"].apply(obtain_token_decimals))
final_df["bridged_amount"] = list(map(lambda x, y: x / y, raw_amount, decimals))
final_df["destination_chain"] = raw_data["transferSentToL2S_destinationChainId"].apply(obtain_chain_name)

In [18]:
# receive token prices from Flipsidecrypto API
from flipside import Flipside


flipside = Flipside("[api-key]", "https://api-v2.flipsidecrypto.xyz")
sql = """SELECT HOUR::date AS DAYS, AVG(PRICE),
CASE WHEN SYMBOL = 'WETH' THEN 'ETH
' ELSE SYMBOL END AS
TOKEN_NAME
FROM ethereum.core.fact_hourly_token_prices
 WHERE SYMBOL IN ('WETH', 'USDC', 'USDT', 'DAI', 'MATIC', 'WBTC') AND DAYS > '2021-06-16'
 GROUP BY DAYS, SYMBOL"""

query_result_set = flipside.query(sql)

t = list(query_result_set)
daily_prices = t[4][1]
daily_prices = list(map(fix_days, daily_prices))
daily_prices_df = pd.DataFrame(daily_prices, columns=["timestamp", "price", "token_name", "t_index"])
daily_prices_df.drop(["t_index"], axis=1, inplace=True)
daily_prices_df["token_name"] = daily_prices_df["token_name"].replace('\n', '', regex=True)

In [19]:
final_df = pd.merge(final_df, daily_prices_df, on=['timestamp', 'token_name'], how='inner')
final_df["bridged_amount_usd"] = final_df["price"].multiply(final_df["bridged_amount"])

# **Querying data**

In [20]:
import  duckdb


*   Total volume of bridged tokens
*   Total number of transactions
*   Total number of addresses

In [41]:
totals = duckdb.sql('SELECT COUNT(distinct( sender)) AS number_of_users, COUNT(DISTINCT (tx_hash)) AS number_of_transactions,'
           ' sum(bridged_amount_usd) AS volume_of_bridged_tokens '
           'From final_df ').to_df().to_json(orient='records', path_or_buf="totals.json")

*   Weekly volume of bridged tokens based on destination chain
*   Weekly number of transactions based on destination chain
*   Weekly number of addresses based on destination chain



In [42]:
daily_destination_chain = duckdb.sql('SELECT date_trunc(\'week\', timestamp::date) as weeks, destination_chain, COUNT(distinct( sender)) AS number_of_addresses,'
            ' COUNT(DISTINCT (tx_hash)) AS number_of_transactions,' ' sum(bridged_amount_usd) AS volume_of_bridged_tokens '
           'From final_df'
            ' GROUP BY weeks, destination_chain order by weeks asc').to_df().to_json(orient='records', date_unit='s', date_format = 'iso', path_or_buf="daily_destination_chain.json")

*   Weekly volume of bridged tokens based on token name
*   Weekly number of transactions based on token name
*   Weekly number of addresses based on token name

In [43]:
daily_token_name = duckdb.sql('SELECT date_trunc(\'week\', timestamp::date) as weeks, token_name, COUNT(distinct( sender)) AS number_of_addresses,'
            ' COUNT(DISTINCT (tx_hash)) AS number_of_transactions,' ' sum(bridged_amount_usd) AS volume_of_bridged_tokens '
           'From final_df'
            ' GROUP BY weeks, token_name order by weeks asc').to_df().to_json(orient='records', date_unit='s', date_format = 'iso', path_or_buf="daily_token_name.json")

Total volume of bridges based on destination chain

In [44]:
volume_of_bridges_based_on_destination_chain = duckdb.sql('SELECT  destination_chain, sum(bridged_amount_usd) AS volume_of_bridged_tokens '
           'From final_df group by destination_chain').to_df().to_json(orient='records', path_or_buf="volume_of_bridges_based_on_destination_chain.json")

Total volume of bridges based on token name

In [45]:
volume_of_bridges_based_on_token = duckdb.sql('SELECT  token_name, sum(bridged_amount_usd) AS volume_of_bridged_tokens '
           'From final_df '
            'group by token_name order by  volume_of_bridged_tokens desc ').to_df().to_json(orient='records', path_or_buf = "volume_of_bridges_based_on_token.json")

Total number of transactions based on destination chain

In [46]:
number_of_transactions_based_on_destination_chain = duckdb.sql('SELECT  destination_chain, count(distinct (tx_hash)) AS number_of_transactions '
           'From final_df group by destination_chain').to_df().to_json(orient='records', path_or_buf="number_of_transactions_based_on_destination_chain.json")

Total number of transactions based on token name


In [47]:
number_of_transactions_based_on_token_name = duckdb.sql('SELECT  token_name, count(distinct (tx_hash)) AS number_of_transactions '
           'From final_df group by token_name').to_df().to_json(orient='records', path_or_buf="number_of_transactions_based_on_token_name.json")

Total number of addresses based on destination chain

In [48]:
number_of_addresses_based_on_destination_chain = duckdb.sql('SELECT  destination_chain, count(distinct (sender)) AS number_of_addresses '
           'From final_df group by destination_chain').to_df().to_json(orient='records', path_or_buf="number_of_addresses_based_on_destination_chain.json")

Total number of addresses based on token name

In [49]:
number_of_addresses_based_on_token_name = duckdb.sql('SELECT  token_name, count(distinct (sender)) AS number_of_addresses '
           'From final_df group by token_name').to_df().to_json(orient='records', path_or_buf="number_of_addresses_based_on_token_name.json")

Growth of volume of bridges

In [58]:
growth_of_volume_of_bridges = duckdb.sql('SELECT  timestamp ::date as days, sum(bridged_amount_usd) AS volume_of_bridged_tokens,'
                                         ' sum(volume_of_bridged_tokens) OVER (ORDER BY days)  as growth_of_volume_of_bridged_tokens '
           'From final_df group by days '
            'order by days asc ').to_df().to_json(orient='records', date_unit='s', date_format = 'iso' ,path_or_buf="growth_of_volume_of_bridged_tokens.json")

Growth of number of transactions



In [59]:
growth_of_number_of_transactions = duckdb.sql('SELECT  timestamp ::date as days, count(distinct (tx_hash))AS number_of_transactions,'
                                         ' sum(number_of_transactions) OVER (ORDER BY days)  as growth_of_number_of_transactions '
           'From final_df group by days '
            'order by days asc ').to_df().to_json(orient='records', date_unit='s', date_format = 'iso' ,path_or_buf="growth_of_number_of_transactions.json")

Growth of number of addresses


In [60]:
first_activity = duckdb.sql('SELECT  min(timestamp) ::date as days, sender '
           'From final_df group by sender '
            'order by days asc ')
#%%
growth_of_number_of_addresses = duckdb.sql('SELECT  days, count(distinct (sender)) number_of_addresses ,  sum(number_of_addresses) OVER (ORDER BY days)  as growth_of_number_addresses '
           'From first_activity group by days').to_df().to_json(orient='records', date_unit='s', date_format = 'iso' ,path_or_buf="growth_of_number_of_addresses.json")

Top 10 addresses based on the volume of bridges

In [53]:
Top_addresses_based_on_the_volume_of_bridges = duckdb.sql('SELECT  sender, sum(bridged_amount) as volume '
           'From final_df group by sender order by volume desc limit 10').to_df().to_json(orient='records', path_or_buf="Top_addresses_based_on_the_volume_of_bridges.json")

Top 10 addresses based on the number of transactions

In [54]:
Top_addresses_based_on_the_number_of_transactions = duckdb.sql('SELECT  sender, count(distinct(tx_hash)) as tx_number '
           'From final_df group by sender order by tx_number desc limit 10').to_df().to_json(orient='records', path_or_buf="Top_addresses_based_on_the_number_of_transactions.json")